In [2]:
from transformers import AutoTokenizer, AutoModel, AutoModelForTokenClassification,pipeline
import pandas as pd
import os
import xmltodict
import requests
import xml.etree.ElementTree as ET
import json

# OpenAccess Extraction

In [ ]:
os.environ['TRANSFORMERS_CACHE'] = './.cache'


def read_text_files_to_df(folder_path, encoding='UTF-8'):
    file_list = []
    for file in os.listdir(folder_path):
        if file.endswith(".txt"):
            file_list.append(os.path.join(folder_path, file))
    
    data = []
    for file in file_list:
        try:
            with open(file, 'r', encoding=encoding) as f:
                data.append(f.read())
        except UnicodeDecodeError:
            with open(file, 'r', encoding='ISO-8859-1') as f:
                data.append(f.read())
    
    df = pd.DataFrame({'content': data})
    return df

def check_keywords(row):
    for keyword in keywords:
        if keyword not in row['content']:
            return False
    return True

def NER_per_text(df, nlp, i, threshold):
    start = 0 
    end = 512

    sentence = df['content'][i][start:end]
    result = nlp(sentence)
    final = pd.DataFrame(result)

    while end < len(df['content'][i]):
        start = end +1
        end = start + 512
        if end > len(df['content'][i]):
            end = len(df['content'][i])
        sentence = df['content'][i][start:end]
        result = pd.DataFrame(nlp(sentence))
        if not result.empty:
            result['start'] +=  start
            result['end'] += start
        final = final.append(result, ignore_index=True)
    final = final[final['score'] >= threshold]
    final = final.sort_values(by=['start'])
    return final

def df_cleaning(df):
    new_entries = []
    ends = []
    starts = []
    entities = []
    for i, row in df.iterrows():
        # if the entry starts with '##', combine it with the previous entry
        if row['word'].startswith('##'):
            try:
                new_entries[-1] = new_entries[-1].strip() + row['word'][2:].strip()
            except:
                continue
            ends[-1] = row['end']
        else:
            new_entries.append(row['word'].strip())
            ends.append(row['end'])
            starts.append(row['start'])
            entities.append(row['entity'])
    concatenated_text = []
    if not new_entries:
        return(pd.DataFrame())
    current_text = new_entries[0]
    current_start = starts[0]
    current_end = ends[0]
    current_ent = entities[0]
    for i in range(1, len(new_entries)):
        if starts[i] == current_end+1 or starts[i] == current_end:
            current_text = current_text + " " +new_entries[i]
            current_end = ends[i]
            current_ent = entities[i]
        else:
            concatenated_text.append((current_ent, current_text, current_start, current_end))
            current_text = new_entries[i]
            current_start = starts[i]
            current_end = ends[i]
            current_ent = entities[i]
    concatenated_text.append((current_ent, current_text, current_start, current_end))
    return pd.DataFrame(concatenated_text, columns=['entity','word', 'start', 'end'])


In [ ]:
##Corpus Aggregation
df1 = read_text_files_to_df("PMC001xxxxxx")
df2 = read_text_files_to_df("PMC002xxxxxx")
df3 = read_text_files_to_df("PMC003xxxxxx")
df4 = read_text_files_to_df("PMC004xxxxxx")
df5 = read_text_files_to_df("PMC005xxxxxx")
df6 = read_text_files_to_df("PMC006xxxxxx")
df7 = read_text_files_to_df("PMC007xxxxxx")
df8 = read_text_files_to_df("PMC008xxxxxx")
df9 = read_text_files_to_df("PMC009xxxxxx")
#df = pd.concat([df1, df2, df3, df4, df5, ])
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9])
keywords = ['cancer', 'histone modifier']
df = df[df.apply(check_keywords, axis=1)]


In [ ]:
result_dict = dict()
#tokenizer = AutoTokenizer.from_pretrained("drAbreu/bioBERT-NER-BC2GM_corpus")
disease_tokenizer = AutoTokenizer.from_pretrained("alvaroalon2/biobert_diseases_ner")
disease_model = AutoModelForTokenClassification.from_pretrained("alvaroalon2/biobert_diseases_ner")
genetic_tokenizer = AutoTokenizer.from_pretrained("alvaroalon2/biobert_genetic_ner")
genetic_model = AutoModelForTokenClassification.from_pretrained("alvaroalon2/biobert_genetic_ner")
disease_nlp = pipeline("ner", model=disease_model, tokenizer=disease_tokenizer)
genetic_nlp = pipeline("ner", model=genetic_model, tokenizer=genetic_tokenizer)

for i in df.index:
    temp_df = NER_per_text(df,disease_nlp,i,0.85)
    temp_df = temp_df[temp_df['entity'] != '0']
    temp_df = df_cleaning(temp_df)
    temp_df = temp_df.append(df_cleaning(NER_per_text(df,genetic_nlp,i,0.85)))
    temp_df.sort_values(by=['start'])
    result_dict[i] = temp_df
ent = []
for i in result_dict.items():
    for j in (i[1]['word']):
        ent.append(j)
ent = set(ent)

In [ ]:
len(list_)

In [ ]:
set(flat_list)

### 588 Full articles

In [ ]:
len(success)

In [ ]:
len(fail)

In [ ]:
response_json= get_json('10.1016/j.cell.2022.05.010')

In [ ]:
print(response_json['full-text-retrieval-response']['originalText']['xocs:doc']['xocs:serial-item']['simple-article'].keys())

In [ ]:
'article' in response_json['full-text-retrieval-response']['originalText']['xocs:doc']['xocs:serial-item']

In [ ]:
list_text

In [ ]:
response_json['full-text-retrieval-response']['originalText']['xocs:doc']['xocs:serial-item']['simple-article']['body']['ce:sections']['ce:section'][0]['ce:section'][0]['ce:para'][0]



In [ ]:
fail

In [ ]:
response_json['full-text-retrieval-response']['coredata']['dc:description'] ##Abstract

In [ ]:
response_json['full-text-retrieval-response']['coredata']['dcterms:subject'] ##Relevant topics -- Screen here for histones?

In [ ]:
##response_json['full-text-retrieval-response']['originalText']['xocs:doc']['xocs:serial-item']['article']['head'] 
##Bolded "abstract/intro" text and data availability statements

In [ ]:
##response_json['full-text-retrieval-response']['originalText']['xocs:doc']['xocs:serial-item']['article']['tail'] 
##Citations


In [ ]:
##response_json['full-text-retrieval-response']['originalText']['xocs:doc']['xocs:serial-item']['article']['body']['ce:appendices'] 
##No supplemental text itself, may not be necessary

In [ ]:
#response_json['full-text-retrieval-response']['originalText']['xocs:doc']['xocs:serial-item']['article']['body']['ce:sections']['ce:section'][1]['ce:section'][0]
#Not all sections structured the same



In [ ]:
NER_results(nlps,string_result)

In [ ]:
input_df = input_df[input_df["Open Access"].isna()]
disease_tokenizer = AutoTokenizer.from_pretrained("alvaroalon2/biobert_diseases_ner")
disease_model = AutoModelForTokenClassification.from_pretrained("alvaroalon2/biobert_diseases_ner")
genetic_tokenizer = AutoTokenizer.from_pretrained("alvaroalon2/biobert_genetic_ner")
genetic_model = AutoModelForTokenClassification.from_pretrained("alvaroalon2/biobert_genetic_ner")
disease_nlp = pipeline("ner", model=disease_model, tokenizer=disease_tokenizer)
genetic_nlp = pipeline("ner", model=genetic_model, tokenizer=genetic_tokenizer)
nlps=[disease_nlp,genetic_nlp]

In [230]:
def flatten(container):
    for i in container:
        if isinstance(i, (list,tuple)):
            for j in flatten(i):
                yield j
        else:
            yield i 

def getKey(d, key):
    res_list = []
    fin_list = []
    try:
        res_list.append(d[key])
    except:
        try:
            for i in d.keys():
                res_list.append(getKey(d[i], key))
        except:
            if type(d) is list:
                for i in d:
                    if type(i) is dict:
                        res_list.append(getKey(i, key))
    for i in res_list:
        if i:
            fin_list.append(i)
    return(fin_list)

def getNatureArticles(doi):
    api_key = '499d84c073f0ade469f211fd37104d7d'
    query = f'https://spdi.public.springernature.app/xmldata/jats?q=doi:{doi}&api_key={api_key}/wustl-api'
    response = requests.get(query)
    dict_data = xmltodict.parse(response.content)
    nature_article = (list(flatten(getKey(dict_data['response']['records']['article']['body'],'#text'))))
    return(nature_article)

In [ ]:
success = []
fail = []
for i in input_df['DOI']:
    print(f'\r{i}', end='',)
    try:
        string_result = ''.join(getNatureArticles(doi))
        success.append(NER_results(nlps,string_result))
    except:
        fail.append(i)



In [ ]:
''.join(getNatureArticles(doi))

In [22]:
#dict_data['response']['records']['article']['@article-type'] ##Could be useful for screening

In [ ]:
#dict_data['response']['records']['article']['front'] ## Author/institutions etc

In [ ]:
#dict_data['response']['records']['article']['back'] ## supplementary/acknowledgements/citations